# Project : Voice Biometric And Face Recognition System For Security


## POC     : Consider there is a door with access control which can be unlocked with your voice and face.

### Tasks: 
### 1) Prepare database/model with your face and voice data.
### 2) Identifies your voice with existing database/model.
### 3) Identifies your face with existing database/model.
### 4) Successful in order to unlock the door or keep lock.





In [ ]:
# import required packages
# for speech recognition
import speech_recognition as sr

# for mysql database / model 
import mysql.connector as msqlconn

# for image processing  
import cv2

# for face recognition
import face_recognition

In [ ]:
# function for recognize speech using Google Speech
def googlespeech_speech_recognize_from_mic(recognize,audio):
    name = ""
    try:
        print("Google Speech thinks your name is : " + recognize.recognize_google(audio)+" \n ")
        name = recognize.recognize_google(audio)
    except sr.UnknownValueError:
        print("Google Speech could not understand audio. Please try again.\n")
        name = ""
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e)+"\n")
        name = ""
    return name

In [ ]:
# function for identifies your voice with existing database/model
def recognize_voice_from_mic(text,mycursor):
    imgPath = ""
    try:
        # remove spaces from voice
        text = text.replace(" ","")
        # convert voice into lower case
        text = text.lower()
        
        # printing query for debug info
        print("\t[Info] : SELECT name,image FROM user WHERE LOWER(name)='"+text+"'\n")
        # find voice into existing database model
        mycursor.execute("SELECT name,image FROM user WHERE LOWER(name)='"+text+"'")
        
        # retrive all the voice match users
        myresult = mycursor.fetchall()
        
        # in our database base one user => one voice
        # check how many users return, if return one user then we process
        if mycursor.rowcount == 1 :
            
            # get user details
            userRecord = myresult[0];
            # extract name from user details
            name = userRecord[0]
            # extract image name from user details
            imageName = userRecord[1]
            
            # if extra spaces exists then remove 
            imageName = imageName.replace(" ", "")
            # preparing image path
            imgPath = "C:/Users/Pritul/Pictures/Camera Roll/"+imageName+""
            
            # printing image path for debug info
            print("\t[Info] : "+imgPath+"\n")
            print("Congratulations, Your voice successfully recognized.\n")
        else:
            print("Sorry, Please Re-Run program and speak your valid name.\n")
            imgPath = ""
    except Exception as e:
        print("Exception occured while recognizing voice.\n")
        imgPath = ""
    return imgPath

In [ ]:
# function for sppech recognize
def recognize_face_from_webcam(imgPath):
    status = ""
    try:
        # get webcam #0 (the default one)
        video_capture = cv2.VideoCapture(0)
        
        # image traning phase
        # load a image and learn how to recognize it.
        existing_image = face_recognition.load_image_file(imgPath)
        # encoded the loaded image into a feature vector
        existing_face_encoding = face_recognition.face_encodings(existing_image)[0]

        # create arrays of known face encodings and their names
        known_face_encodings = [
            existing_face_encoding
        ]

        while True:

            # get one by one frame of current video (one by one images)
            ret, frame = video_capture.read()
            
            # convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
            # rgb values between 0 to 255
            rgb_frame = frame[:, :, ::-1]

            # testing phase
            # find all the faces
            face_locations = face_recognition.face_locations(rgb_frame)

            # find all the faces and face encodings in the current frame of video
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            # iterate one by one face from video frame (images)
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):

                # check face is match for voice match user
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                #print(matches)

                # if face match then print message,
                if True in matches:
                    print("Congratulations, Your face successfully recognized. Exit program: click on webcam viceo then Press (q) key \n")
                    status = "Congratulations, Your face successfully recognized. Exit program: click on webcam viceo then Press (q) key \n"
                else:
                    print("Sorry, Please match your valid face. Exit program: click on webcam viceo then Press (q) key \n")
                    status = "Sorry, Please match your valid face. Exit program: click on webcam viceo then Press (q) key \n"

       
            # display the resulting image
            cv2.imshow('Video', frame)

            # Hit 'q' on the keyboard to quit!
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Release handle to the webcam
        video_capture.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print("Exception occured while recognizing face.\n")
        print(e)
        status = ""
    return status

In [ ]:
mydb = ""

mycursor = ""

try:
    # connect MySQL server and returns a MySQLConnection object. 
    mydb = msqlconn.connect(host="localhost",user="root",passwd="",database="ai_demo")

    # Create a new cursor
    mycursor = mydb.cursor()
    
    # Create recognizer object to recognize our voice
    recognize = sr.Recognizer()

    # get the from Microphone 
    with sr.Microphone() as source:

        #Print message for user, user know what I need to do
        print("Speak your name only :\n")

        # adjust the recognizer sensitivity to ambient noise and record audio from the microphone
        recognize.adjust_for_ambient_noise(source)

        #Using listen() to capture mic input
        audio = recognize.listen(source)

        # set the empty default value
        text = ""

        # # get text from mic using google sevice
        text = googlespeech_speech_recognize_from_mic(recognize,audio)
        text = text.strip()

        # if text == empty then try with google sevice   
        if len(text) == 0 :
            # get text from mic using google sevice
            text = googlespeech_speech_recognize_from_mic(recognize,audio)
            text = text.strip()
            if len(text) != 0 :
                # match voice with database model
                imgPath = recognize_voice_from_mic(text,mycursor)
                imgPath = imgPath.strip()
                if len(imgPath) != 0 :
                     # match face with database model
                    status = recognize_face_from_webcam(imgPath)
                    status = status.strip()
                    if len(status) != 0 :
                        print(status)     
        else :
            # match voice with database model
            imgPath = recognize_voice_from_mic(text,mycursor)
            imgPath = imgPath.strip()
            if len(imgPath) != 0 :
                 # match face with database model
                status = recognize_face_from_webcam(imgPath)
                status = status.strip()
                if len(status) != 0 :
                    print(status)
except Exception as e:
    print("Exception occured while running main module.\n")
finally:
    mydb.close()
    mycursor.close()